In [1]:
import setGPU

import os
import time

import tensorflow as tf
from tensorflow.python.client import device_lib

setGPU: Setting GPU to: 1


/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3

In [2]:
#print(tf.config.experimental.list_physical_devices('GPU'))
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16949033818433704111
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13798967128644991299
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10813738189
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3313777990042199390
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:b3:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 477417605264542908
physical_device_desc: "device: XLA_GPU device"
]


In [2]:
tf.enable_eager_execution()

In [2]:
print(tf.__version__)

1.14.0


In [3]:
_BATCH_SIZE = 128
_EPOCHS = 50
_NUM_CLASSES = 76
img_width, img_height = 299, 299

In [4]:
def generate_train_valid_lists(train_path, valid_path):
        train_files = []
        train_labels = []
        
        #generating lists of train dataset
        for title in os.listdir(train_path):
            title_path = train_path + title
            for fname in os.listdir(title_path):
                file_path = title_path + '/' + fname
                train_files.append(file_path)
                title = int(title)
                train_labels.append(title)
                
        #generating lists of validation dataset
        valid_files = []
        valid_labels = []
        for title in os.listdir(valid_path):
            title_path = valid_path + title
            for fname in os.listdir(title_path):
                file_path = title_path + '/' + fname
                valid_files.append(file_path)
                title = int(title)
                valid_labels.append(title)
                
        return (train_files, train_labels), (valid_files, valid_labels)

In [5]:
def generate_label_index(labels):
    init_label = labels[0]
    label_index = 1
    label_index_list = []
    #title_to_index_map = {}
    for label in labels:
        if label != init_label:
            init_label = label
            label_index += 1
        label_index_list.append(label_index)
        #title_to_index_mapappend=(dict(label, label_index))
            
    return label_index_list

In [7]:
"""
Generate file names of training and validation as lists
"""
train_path = '../../../data/movie_titles/mini_data/train/'
valid_path = '../../../data/movie_titles/mini_data/validation/'

(train_files, train_labels), (valid_files, valid_labels) = generate_train_valid_lists(train_path, valid_path)
len_train_samples = len(train_files)
len_valid_samples = len(valid_files)
print("train data info:", len(train_files), len(train_labels), type(train_files[0]), type(train_labels[0]))
print("valid data info:", len(valid_files), len(valid_labels))

"""
Map train and validation labels to 1 to 76 range from title names
"""
train_label_in_classes_range = generate_label_index(train_labels)
valid_label_in_classes_range = generate_label_index(valid_labels)
#print(train_label_in_classes_range[:100000])

train data info: 66194 66194 <class 'str'> <class 'int'>
valid data info: 19330 19330


In [8]:
"""
Experimenting range values with single image
"""
img_raw = tf.io.read_file(train_files[0])
img_tensor = tf.image.decode_jpeg(img_raw)
print(img_tensor.shape)
print(img_tensor.dtype)
img_final = tf.image.resize_images(img_tensor, [img_width, img_height])
#img_final = tf.image.convert_image_dtype(img_final, tf.float32)
#img_final /= 255.0
#img_final = 2*(img_final/255.0)-1.0
img_final = tf.keras.applications.inception_v3.preprocess_input(img_final)
print(img_final.shape)

(?, ?, ?)
<dtype: 'uint8'>
(299, 299, ?)


In [9]:
def _load_image(image, label):
    img_string = tf.read_file(image)
    img_decoded = tf.image.decode_jpeg(img_string, channels=3)
    resized_img = tf.image.resize_images(img_decoded, [img_width, img_height])
    img_final = tf.keras.applications.inception_v3.preprocess_input(resized_img)
    label = tf.one_hot(tf.cast(label, tf.uint8), _NUM_CLASSES)
    return img_final, label

In [10]:
def _load_images(image):
    img_string = tf.read_file(image)
    img_decoded = tf.image.decode_jpeg(img_string, channels=3)
    resized_img = tf.image.resize_images(img_decoded, [img_width, img_height])
    img_final = tf.image.convert_image_dtype(resized_img, tf.float32)
    img_final = tf.keras.applications.inception_v3.preprocess_input(img_final)
    return img_final

In [11]:
def load_preprocess_from_path_label(path, label):
    label = tf.one_hot(tf.cast(label, tf.uint8), _NUM_CLASSES)
    return _load_images(path), label

In [ ]:
"""
viewing a couple of image on train dataset and verify them
"""
import matplotlib.pyplot as plt
path_ds = tf.data.Dataset.from_tensor_slices((train_files, train_label_in_classes_range))
#path_ds = tf.data.Dataset.from_tensor_slices(train_files)
image_ds = path_ds.map(_load_image, num_parallel_calls=4)

plt.figure(figsize=(8,8))
for n, image in enumerate(image_ds.take(4)):
    #plt.subplot(2, 2, n+1)
    #plt.imshow(image)
    print(image[1])

Tensor("IteratorGetNext:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_1:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_2:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_3:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_4:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_5:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_6:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_7:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_8:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_9:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_10:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_11:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_12:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_13:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_14:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_15:1", shape=(76,), dtype=float32)
Tensor("IteratorGetNext_16:1", shape=(76,), dtype=float32)
Tensor("I

In [14]:
def display_img_and_label():
    len_img = len(train_files)
    len_lab = len(train_labels)
    
    #print(len_img, len_lab)
    
    for i in range(0, len_img):
        if not str(int(train_labels[i])) in train_files[i]:
            print(train_labels[i], train_files[i])

In [15]:
"""
Display image path and associated label
"""
display_img_and_label()

In [11]:
#create tf tensors of list of files

train_files_tensor = tf.constant(train_files)
#train_labels_tensor = tf.constant(train_labels, dtype=tf.float32)
train_labels_tensor = tf.constant(train_label_in_classes_range)
print(train_labels_tensor)

valid_files_tensor = tf.constant(valid_files)
#valid_labels_tensor = tf.constant(valid_labels)
valid_labels_tensor = tf.constant(valid_label_in_classes_range)

Tensor("Const_1:0", shape=(38000,), dtype=int32)


In [12]:
def tfdata_generator(images, labels, is_training=True, batch_size=64):
       
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if is_training:
        dataset = dataset.shuffle(len_train_samples) #depends on sample size
        
    #dataset = dataset.apply(tf.data.experimental.map_and_batch(_load_image, batch_size, num_parallel_batches=4, drop_remainder=True if is_training else False))
    dataset = dataset.repeat()
    dataset = dataset.map(load_preprocess_from_path_label, num_parallel_calls=4)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.contrib.data.AUTOTUNE)
    
    return dataset

In [24]:
def get_base_model(img_shape, is_inception=True):
    if is_inception:
        conv_base_model = tf.keras.applications.InceptionV3(input_shape=img_shape, include_top=False, weights='imagenet')
    else:
        conv_base_model = tf.keras.applications.VGG16(input_shape=img_shape, include_top=False, weights='imagenet')
        
    #conv_base_model.summary()
    
    return conv_base_model

In [40]:
def training_pipeline():
    train_set = tfdata_generator(train_files_tensor, train_labels_tensor, is_training=True, batch_size=_BATCH_SIZE)
    valid_set = tfdata_generator(valid_files_tensor, valid_labels_tensor, is_training=False, batch_size=_BATCH_SIZE)
    #print(train_set)
    #print(valid_set)
    
    img_shape = (img_width, img_height, 3)
    base_model = get_base_model(img_shape)
    
    head_model = base_model.output
    head_model = tf.keras.layers.Flatten(name='flatten')(head_model)
    head_model = tf.keras.layers.Dense(1024, activation='relu', name='dense1')(head_model)
    head_model = tf.keras.layers.Dense(512, activation ='relu', name='dense2')(head_model)
    head_model = tf.keras.layers.Dense(128, activation='relu', name='dense3')(head_model)
    head_model = tf.keras.layers.Dense(_NUM_CLASSES, activation='softmax', name='final_predictions')(head_model)
    
    model = tf.keras.Model(inputs=base_model.inputs, outputs=head_model)
    print(len(model.trainable_variables))
    for layer in base_model.layers:
        layer.trainable = False
    print(len(model.trainable_variables))
    #model.summary()

    #model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    """
    iterator = train_set.make_one_shot_iterator()
    images, labels = iterator.get_next()
    
    iterator_v = valid_set.make_one_shot_iterator()
    v_images, v_labels = iterator_v.get_next()
    
    with tf.Session() as sess:
        imgs = sess.run(images)
        labls = sess.run(labels)
        print(imgs.shape, labls.shape)
        
        v_imgs = sess.run(v_images)
        v_lbls = sess.run(v_labels)
        print(v_imgs.shape, v_lbls.shape)"""
    
    #model.fit(train_set.make_one_shot_iterator(), steps_per_epoch=len_train_samples //_BATCH_SIZE, epochs=_EPOCHS, validation_data=valid_set.make_one_shot_iterator(), 
    #          validation_steps=len_valid_samples //_BATCH_SIZE, verbose=1)

In [41]:
training_pipeline()

196
8
